In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function, division
import torch as tr
import os, argparse, logging, json, pickle
from configs import Config
from exp_context import ExperimentContext
from base.hyperparams import Hyperparams as H

print('mode:', 'gpu' if Config.use_gpu else 'cpu')

if Config.use_gpu:
    tr.set_default_tensor_type('torch.cuda.FloatTensor')

args_str = '-hp base/hyperparams.py -d all -en exp14_node_split'

mode: gpu


#### **Argument Parser**

In [3]:
parser = argparse.ArgumentParser()

parser.add_argument('-g', '--gpu', default=0, help='index of the gpu to be used. default: 0')
parser.add_argument('-r', '--resume', nargs='?', const=True, default=False,
                    help='if present, the training resumes from the latest step, '
                         'for custom step number, provide it as argument value')
parser.add_argument('-d', '--delete', nargs='+', default=[], choices=['logs', 'weights', 'results', 'all'],
                    help='delete the entities')
parser.add_argument('-w', '--weights', nargs='?', default='iter', choices=['iter', 'best_gen', 'best_pred'],
                    help='weight type to load if resume flag is provided. default: iter')
parser.add_argument('-hp', '--hyperparams', required=True, help='hyperparam class to use from HyperparamFactory')
parser.add_argument('-en', '--exp_name', default=None, help='experiment name. if not provided, it is taken from Hyperparams')

args = parser.parse_args(args_str.split())
print(json.dumps(args.__dict__, indent=4))

{
    "resume": false, 
    "exp_name": "exp14_node_split", 
    "hyperparams": "base/hyperparams.py", 
    "weights": "iter", 
    "gpu": 0, 
    "delete": [
        "all"
    ]
}


In [4]:
resume_flag = args.resume is not False
gpu_idx = str(args.gpu)
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_idx

#### **Set Experiment Context**

In [5]:
ExperimentContext.set_context(args.hyperparams, args.exp_name)
H = ExperimentContext.Hyperparams  # type: Hyperparams

loading HP from file
importing hyperparams base.hyperparams


#### **Set Logging**

In [6]:
logger = logging.getLogger(__name__)
LOG_FORMAT = "[{}: %(filename)s: %(lineno)3s] %(levelname)s: %(funcName)s(): %(message)s".format(ExperimentContext.exp_name)
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

#### **Clear Logs and Results based on the argument flags**

In [7]:
from paths import Paths
from utils import bash_utils, model_utils

if 'all' in args.delete or 'logs' in args.delete or resume_flag is False:
    logger.warning('Deleting Logs...')
    bash_utils.delete_recursive(Paths.logs_base_dir)
    print('')

if 'all' in args.delete or 'results' in args.delete:
    logger.warning('Deleting all results in {}...'.format(Paths.results_base_dir))
    bash_utils.delete_recursive(Paths.results_base_dir)
    print('')

[exp14_node_split: <ipython-input-7-a00023a52412>:   5] WARNING: <module>(): Deleting Logs...
[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): rm -r ../experiments/exp14_node_split/logs
[exp14_node_split: <ipython-input-7-a00023a52412>:  10] WARNING: <module>(): Deleting all results in ../experiments/exp14_node_split/results...
[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): rm -r ../experiments/exp14_node_split/results


##### **Create required directories**

In [8]:
model_utils.setup_dirs()

[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): mkdir -p ../experiments/exp14_node_split/results
[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): mkdir -p ../experiments/exp14_node_split/logs
[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): mkdir -p ../experiments/exp14_node_split/weights/saved/
[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): mkdir -p ../experiments/exp14_node_split/weights/all/
[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): mkdir -p ../experiments/exp14_node_split/results
[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): mkdir -p ../experiments/exp14_node_split/results/.temp


##### **Model and Training related imports**

In [9]:
from dataloaders.factory import DataLoaderFactory
from base.hyperparams import Hyperparams

from models.toy.gan import ToyGAN
from models.toy.gt.gantree import GanTree
from models.toy.gt.gnode import GNode
from models.toy.gt.utils import GNodeUtils, DistParams

from trainers.gan_trainer import GanTrainer
from trainers.gan_trainer import TrainConfig

##### **Tensorboard Port**

In [10]:
ip = bash_utils.get_ip_address()
tboard_port = str(bash_utils.find_free_port(Config.base_port))
bash_utils.launchTensorBoard(Paths.logs_base_dir, tboard_port)
address = '{ip}:{port}'.format(ip=ip, port=tboard_port)
address_str = 'http://{}'.format(address)
tensorboard_msg = "Tensorboard active at http://%s:%s" % (ip, tboard_port)
html_content = """
<h5>
    <b>Tensorboard hosted at 
        <a href={}>{}</a>
    </b>
</h5>
""".format(address_str, address)
from IPython.core.display import display, HTML
display(HTML(html_content))

##### **Dump Hyperparams file the experiments directory**

In [11]:
hyperparams_string_content = json.dumps(H.__dict__, default=lambda x: repr(x), indent=4, sort_keys=True)
# print(hyperparams_string_content)
with open(Paths.exp_hyperparams_file, "w") as fp:
    fp.write(hyperparams_string_content)

##### **Define Train Config**

In [12]:
train_config = TrainConfig(
    n_step_tboard_log=50,
    n_step_console_log=-1,
    n_step_validation=100,
    n_step_save_params=1000,
    n_step_visualize=500
)

##### **Create Gan Model and DataLoader for root GNode**

In [13]:
gan = ToyGAN.create_from_hyperparams('node0', H, '+')
dist_params = DistParams(gan.z_op_params[0], gan.z_op_params[1], 1.0, 1.0)
dl = DataLoaderFactory.get_dataloader(H.dataloader, H.input_size, H.z_size, H.batch_size, H.batch_size, supervised=True)
x_batch, _ = dl.random_batch('test', 2048)

##### **Create Gan Tree and GNode**

In [14]:
tree = GanTree('gtree', ToyGAN, H, x_batch)
gnode = tree.create_child_node(dist_params, gan)

[exp14_node_split: utils.py:  44] INFO: create_child_node(): Child Node node0 created
[exp14_node_split: utils.py:  45] INFO: create_child_node(): Node parameters: 
[exp14_node_split: utils.py:  46] INFO: create_child_node(): prior_means: tensor([0., 0.])
[exp14_node_split: utils.py:  47] INFO: create_child_node(): prior_cov  : tensor([[1.0000, 0.6000],
        [0.6000, 1.0000]])
[exp14_node_split: utils.py:  48] INFO: create_child_node(): cond_prob  : 1.0
[exp14_node_split: utils.py:  49] INFO: create_child_node(): abs_prob   : 1.0


##### **Set Trainer for GNode**

In [15]:
gnode.set_trainer(dl, H, train_config)

[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): mkdir -p ../experiments/exp14_node_split/weights/iter/node0
[exp14_node_split: bash_utils.py:  10] INFO: exec_cmd(): mkdir -p ../experiments/exp14_node_split/weights/best/node0


In [16]:
display(HTML(html_content))

In [17]:
# gnode.train(10000)
# gnode.save('split_node.pickle')

In [18]:
gnode = GNode.load('split_node.pickle')

In [19]:
if gnode.is_leaf:
    tree.split_node(gnode, x_batch, fixed=False)
    print ('splitted')

[exp14_node_split: utils.py:  59] INFO: split_node(): Starting Split Process: <GNode[name=node0 id=0 parent_id=-1]>
[exp14_node_split: utils.py:  61] INFO: split_node(): Gaussian Mixture Fitted


RuntimeError: invalid argument 1: expected a matrix at /pytorch/aten/src/THC/generic/THCTensorMath.cu:395

In [ ]:
z = gnode.post_gmm_encode(dl.data['test'])

In [ ]:
from trainers.gnode_trainer import GNodeTrainer
trainer = GNodeTrainer(gnode, dl, H, train_config)
trainer.relabel_data()

In [ ]:
trainer.train(100)
# gnode.child_nodes[2].trainer.data_loader.data['train'].shape

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import style
from matplotlib import pyplot as plt
from utils import viz_utils
from scipy import stats
style.use(style.available[14])

In [ ]:
data = dl.data['train']
zdata = gnode.post_gmm_encode(data)
print(gnode.predict_z(zdata))
label = gnode.predict_x(data)
colors = np.array(['red', 'blue'])[label-1]
plt.scatter(zdata[:, 0], zdata[:, 1])

In [ ]:
x_splits, i_splits = gnode.split_x(data)

In [ ]:
from torch import nn
super(nn.Module, gnode)

In [ ]:
print(gnode.child_nodes)
node1, node2 = gnode.child_nodes.values()
means = node1.prior_means, node2.prior_means
cov = node1.prior_cov, node2.prior_cov

print(means)
print(cov[0])
print(cov[1])

f1 = stats.multivariate_normal(means[0], cov[0])
f2 = stats.multivariate_normal(means[1], cov[1])

ax = plt.gca()
data1, data2 = x_splits[1], x_splits[2]
z1 = node1.gan.encode(data1)
z2 = node2.gan.encode(data2)

print (z1.shape, z2.shape)

print(f1.pdf(z1).mean())
print(f2.pdf(z2).mean())

e1 = viz_utils.get_ellipse(means[0], cov[0], [2], color='pink')
e2 = viz_utils.get_ellipse(means[1], cov[1], [2], color='black')

ax.set_xlim(-7, 7)
ax.set_ylim(-7, 7)
ax.scatter(z1[:, 0], z1[:, 1], color='red', s=1)
ax.scatter(z2[:, 0], z2[:, 1], color='blue', s=1)
for e in e1 + e2:
    ax.add_artist(e)
    
# plt.scatter(data1[:, 0], data1[:, 1], color='red', s=1)
# plt.scatter(data2[:, 0], data2[:, 1], color='blue', s=1)

In [ ]:
means = gnode.gmm.means_
cov = gnode.gmm.covariances_

f1 = stats.multivariate_normal(means[0], cov[0])
f2 = stats.multivariate_normal(means[1], cov[1])

ax = plt.gca()
data1, data2 = x_splits[1], x_splits[2]
z1 = gnode.gan.encode(data1)
z2 = gnode.gan.encode(data2)

print(f1.pdf(z1).mean())
print(f2.pdf(z2).mean())

e1 = viz_utils.get_ellipse(means[0], cov[0], [1, 2, 2.5], color='pink')
e2 = viz_utils.get_ellipse(means[1], cov[1], [1, 2, 2.5], color='blue')

ax.scatter(z1[:, 0], z1[:, 1], color='red', s=1)
ax.scatter(z2[:, 0], z2[:, 1], color='blue', s=1)
for e in e1 + e2:
    ax.add_artist(e)
# for e in e2:
#     ax.add_artist(e)
    
# plt.scatter(data1[:, 0], data1[:, 1], color='red', s=1)
# plt.scatter(data2[:, 0], data2[:, 1], color='blue', s=1)

In [ ]:
gen = gnode.gan.decode(np.random.normal(0, 1, (1000, 2)))

In [ ]:
z_batch = gnode.post_gmm_encode(x_batch)
z_batch_ = gnode.gan.encode(x_batch)

In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture

In [ ]:
gmm = GaussianMixture(2)

In [ ]:
gmm.n_components = 10
A = np.random.uniform(-1, 1, (1000, 2))
B = np.random.normal(1, 1/3.0, (1000, 2))
C = np.concatenate([A, B])
gmm.fit(C)

In [ ]:
gmm.predict_proba(C).shape

In [ ]:
from utils import viz_utils

In [ ]:
%matplotlib inline
import matplotlib.style as style
style.use(style.available[14])
from matplotlib import  pyplot as plt
ax = plt.gca()
ax.scatter(C[:, 0], C[:, 1])

e1 = viz_utils.get_ellipse(gmm.means_[0], gmm.covariances_[0], [1,2,3], color='orange')
e2 = viz_utils.get_ellipse(gmm.means_[1], gmm.covariances_[1], [1,2,3], color='red')

for e in e1 + e2:
    ax.add_artist(e)

In [ ]:
from models.toy import gantree
from models.toy.gantree import GNode

In [ ]:
node = GNode(model=gnode.gan)
node.gmm.fit(dl.data['train'])
node1, node2 = gantree.split_node(node, 2, x_batch, 1)
x_splits, i_splits = node.split_x(dl.data['train'])

In [ ]:
len(x_splits[1]), len(x_splits[2])
node1.dist_params

In [ ]:
plt.scatter(x_splits[1][:, 0], x_splits[1][:, 1], color='red', s=1)
plt.scatter(x_splits[2][:, 0], x_splits[2][:, 1], s=1)

In [ ]:
from torch.nn import functional as F
from torch import nn

In [ ]:
loss = nn.BCEWithLogitsLoss()

In [ ]:
def sigmoid_cross_entropy_loss(logits, labels):
    if isinstance(labels, int):
        if labels == 0.:
            labels = tr.zeros_like(logits)
        elif labels == 1.:
            labels = tr.ones_like(logits)

    losses = tr.max(logits, tr.zeros_like(logits)) - logits * labels + tr.log(1 + tr.exp(-tr.abs(logits)))
    return losses.mean()

In [ ]:
logits = tr.rand(1000,)
labels = tr.randint(0, 2, (1000, ))

In [ ]:
sigmoid_cross_entropy_loss(logits, 1.)

In [ ]:
loss(logits, tr.ones_like(logits))